In [ ]:
# Have a look at the duplicate zips --> if zcta different --> dont need to remove

# fill out the 0 to the zip (some sorts of functions)

# Drop redundant columns (2 columns left)

# Do some merging here between zip_zcta and the restaurant tables to show that they can be matched against the zip_code (don't need to export to table)